In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def set_dtypes(dataframe_index):
  csv_dataframe = csv_dataframes[dataframe_index].convert_dtypes()
  csv_dtypes = []

  name_list = [column for column in csv_dataframe.columns]

  for column_index in range (len(csv_dataframe.columns)):
    column = csv_dataframe[name_list[column_index]]
    column_dtype = column.dtype

    for col_index in range(len(csv_dataframes_enum_properties[dataframe_index])):
      if column_index == csv_dataframes_enum_properties[dataframe_index][col_index][0]:
        column_dtype = 'Enum'
    
    if column_dtype == 'string':
      column_dtype = 'Date Time'
      datetime_count = column.str.contains(':').sum()

      if datetime_count < 20:
          column_dtype = 'String'
    elif column_dtype != 'Enum':
        column_dtype = 'Value'
    
    csv_dtypes.append(column_dtype)

  return csv_dtypes

In [ ]:
csv_dataframes_names = ['INOUT.csv', 'INOUTLINE.csv', 'PRODUCTINSTANCE.csv', 'TRANSFER_ITEM.csv', 'TRANSFER_ITEM_D.csv']
csv_dataframes = []
csv_dataframes_dtypes = []

for dataframe_index in range(len(csv_dataframes_names)):
  csv_dataframes.append(pd.read_csv(csv_dataframes_names[dataframe_index],encoding="ISO-8859-1",low_memory=False))
  csv_dataframes_dtypes.append([])
  csv_dataframes_dtypes[dataframe_index].append(set_dtypes(dataframe_index))

IndexError: ignored

In [ ]:
# Enumerate Properties

# csv_dataframes_enum_prperties = [['TRANSFER_ITEM.csv', 'C_DOCSTATUS_ID']]
csv_dataframes_enum_properties = [[[3, 'ISACTIVE', 'Y'], [12, 'C_DOCSTATUS_ID', '3000025', '3000006', '3000018', '6000035'], [21, 'PITYPE', 'l'], [26, 'ACCEPTANCETYPE', 'T', 'H']], 
                                  [[33, 'M_PI_STATUS_ID', '5800001.0', '5800002.0'], [44, 'ACCEPTED', 'Y', 'N'], [51, 'GROUH_DARAEI', 'l']], 
                                  [[14, 'M_PI_STATUS_ID', '1000016.0', '5800001.0', '5800013.0', '1000013.0', '5010002.0', '1000021.0', '5800002.0', '1000014.0', '5010003.0', '1000006.0', '5010008.0', '6000013.0', '6000023.0', '1000022.0', '1000012.0', '1000004.0', '5010007.0', '1000015.0', '1000018.0', '1000020.0', '1000003.0', '1000024.0', '1000037.0', '1000017.0', '1000005.0', '5800010.0', '5800003.0', '6000028.0', '1000023.0', '1000030.0', '1000019.0', '3000025.0'], [31, 'PITYPE', 'b', 'l', 'm', 'i'], [84, 'A_ASSET_ID', '210618919.0', '210618919', '210032630.0'], [87, 'RETURNAMVALTOANBAR', '1.0', '2.0', '3.0', '4.0'], [127, 'ISCONTROLAMVAL', 'Y', 'N']],
                                  [[23, 'C_DOCSTATUS_ID', '3000006.0', '3000007.0', '3000025.0', '268926802.0', '268926803.0', '268926804.0', '268926805.0', '263750590.0']]]


print(len(len(csv_dataframes_enum_properties[dataframe_index])))

NameError: ignored

In [ ]:
def get_properties(dataframe_index, column_index):
  for col_index in range(len(csv_dataframes_enum_properties[dataframe_index])):
      if column_index == csv_dataframes_enum_properties[dataframe_index][col_index][0]:
        enum_properties = csv_dataframes_enum_properties[dataframe_index][col_index][2:]
        return enum_properties

In [ ]:
def accuracy(dataframe_index, column_index):
  column = csv_dataframes[dataframe_index][csv_dataframes[dataframe_index].columns[column_index]]
  column_dtype = csv_dataframes_dtypes[dataframe_index][0][column_index]

  faulty_values_count = column.isna().sum()

  if column_dtype == 'Value':
    numeric_column = pd.to_numeric(column , errors='coerce')
    faulty_values_count = numeric_column.isna().sum()
  elif column_dtype == 'Date Time':
    faulty_values_count += (len(column) - column.str.contains(':').sum())
  elif column_dtype == 'Enum':
    enum_properties = get_properties(dataframe_index, column_index)
    accurate_values_count = 0
    for property_index in range(len(enum_properties)):
      accurate_values_count += column.astype(str).str.contains(enum_properties[property_index]).sum()
    faulty_values_count += (len(column) - accurate_values_count)
  elif column_dtype == 'String':
    numeric_column = pd.to_numeric(column , errors='coerce')
    faulty_values_count = len(column) - (numeric_column.isna().sum() - faulty_values_count)
  
  accuracy = (len(column) - faulty_values_count) / len(column)

  return accuracy

In [ ]:
def completeness(column):
  return (len(column) - column.isna().sum()) / len(column)

In [ ]:
def create_table(dataframe_index):
  name_list = [column for column in csv_dataframes[dataframe_index].columns]
  table_result = pd.DataFrame()

  for column_index in range (len(csv_dataframes[dataframe_index].columns)):
    column = csv_dataframes[dataframe_index][name_list[column_index]]

    attribute_name = name_list[column_index]
    record_count = len(column)
    null_count = column.isna().sum()
    attribute_accuracy = accuracy(dataframe_index, column_index)
    attribute_completeness = completeness(column)
    
    column_result = {'Attribute Name': attribute_name, 'Record Count': record_count, 'Null Count': null_count, 'Accuracy': attribute_accuracy, 'Completeness': attribute_completeness}
    
    table_result = pd.concat([table_result, pd.DataFrame(column_result,index=[column_index])], axis=0)

  return table_result

In [ ]:
for dataframe_index in range(len(csv_dataframes_names)):
  print("***********************************************************************************************")
  print(csv_dataframes_names[dataframe_index] + "Result Table")
  print("***********************************************************************************************")
  print(create_table(dataframe_index))
  print("***********************************************************************************************")